In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-128153")
exp = Experiment(workspace=ws, name="sklearn-experiment")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-128153
Azure region: southcentralus
Subscription id: e8f628b3-bb5b-4edf-947a-8637ca6ea7c2
Resource group: aml-quickstarts-128153


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal,choice
import os


# Specify parameter sampler
param_sampling = RandomParameterSampling( {
        "C": uniform(0.1,10),
        "max_iter": choice(20,50,100)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 1, slack_factor=0.2, delay_evaluation=5)


project_folder = 'training'
if project_folder not in os.listdir():
    os.mkdir(project_folder)

import shutil
shutil.copy('train.py', project_folder)

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = project_folder,entry_script = 'train.py',compute_target = cpu_cluster)                 

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
                                   hyperparameter_sampling = param_sampling,
                                   primary_metric_name = 'Accuracy',
                                   primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                   max_total_runs = 100,
                                   max_concurrent_runs = 4,
                                   policy = policy,
                                   estimator = est)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails
run = exp.submit(hyperdrive_config)
RunDetails(run).show()
run.wait_for_completion(show_output=True)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_b72db127-a841-454b-8878-f1530278212b
Web View: https://ml.azure.com/experiments/sklearn-experiment/runs/HD_b72db127-a841-454b-8878-f1530278212b?wsid=/subscriptions/e8f628b3-bb5b-4edf-947a-8637ca6ea7c2/resourcegroups/aml-quickstarts-128153/workspaces/quick-starts-ws-128153

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-27T22:28:35.606387][API][INFO]Experiment created<END>\n"<START>[2020-11-27T22:28:37.5385586Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2020-11-27T22:28:38.818289][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-11-27T22:28:39.184977][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"


In [ ]:
import joblib
assert(run.get_status() == "Completed")
# Get your best run and save the model from that run.
best_run = run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())
best_run_metrics = best_run.get_metrics()
best_run.get
print('Best accuracy: {}'.format(best_run_metrics['Accuracy']))
print(os.listdir('./outputs'))
print('========================')


In [ ]:
sklearn_model = best_run.register_model(model_name='sklearn-model.joblib', model_path='./outputs/sklearn-model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

from azureml.data.dataset_factory import TabularDatasetFactory
datastore_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=datastore_path,header=True)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    compute_target=cpu_cluster,
    task='classification',
    primary_metric='accuracy',
    training_data= ds,
    label_column_name='y',
    n_cross_validations=5)

In [ ]:
# Submit your automl run
from azureml.core.experiment import Experiment
automl_experiment = Experiment(ws, "automl-experiment")
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()

In [ ]:
# Retrieve and save best automl model.

best_automl_run, auto_ml_fitted_model = automl_run.get_output()
print(best_automl_run)
print(auto_ml_fitted_model)
model = best_automl_run.register_model(model_name='automl-model', model_path='outputs/model.pkl')


In [ ]:
# Delete Compute Cluster
cpu_cluster.delete()